# Creating a dataset class with activations from the hooks and corresponding tokens

In [5]:
%pip install transformer_lens

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from typing import Any

from tqdm import tqdm
import einops
import numpy as np
import torch
from datasets import Dataset, IterableDataset, load_dataset
from numpy.typing import NDArray
from pydantic import BaseModel, ConfigDict
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

# from e2e_sae.types import Samples
# from e2e_sae.models.transformers import SAETransformer
# from e2e_sae.loader import load_tlens_model

from transformer_lens import HookedTransformer

In [3]:
# model = HookedTransformer.from_pretrained("tiny-stories-1M")
# _logits, cache = model.run_with_cache("Some prompt")

In [4]:
# cache['blocks.0.hook_mlp_out'][0][0]
# cache['blocks.0.hook_resid_mid'][0][0]
# cache['blocks.0.hook_mlp_out'][0][0]
# cache['blocks.0.hook_resid_mid'][0][0] + cache['blocks.0.hook_mlp_out'][0][0]
# cache['blocks.0.hook_resid_post'][0][0], \
# cache['blocks.1.hook_resid_pre'][0][0],

In [5]:
# list(cache.keys())

In [6]:
class DatasetConfig(BaseModel):
    model_config = ConfigDict(extra="forbid", frozen=True)
    dataset_name: str
    is_tokenized: bool = True
    tokenizer_name: str
    streaming: bool = True
    split: str
    n_ctx: int
    seed: int = 0
    column_name: str = "input_ids"
    """The name of the column in the dataset that contains the data (tokenized or non-tokenized).
    Typically 'input_ids' for datasets stored with e2e_sae/scripts/upload_hf_dataset.py, or "tokens"
    for datasets tokenized in TransformerLens (e.g. NeelNanda/pile-10k)."""

In [7]:
# def create_data_loader_only_activations(
#     dataset_config: DatasetConfig, batch_size: int, buffer_size: int = 1000, global_seed: int = 0
# ) -> tuple[DataLoader, AutoTokenizer]:
#     """Create a DataLoader for the given dataset.

#     Args:
#         dataset_config: The configuration for the dataset.
#         batch_size: The batch size.
#         buffer_size: The buffer size for streaming datasets.
#         global_seed: Used for shuffling if dataset_config.seed is None.

#     Returns:
#         A tuple of the DataLoader and the tokenizer.
#     """
#     dataset = load_dataset(
#         dataset_config.dataset_name, streaming=dataset_config.streaming, split=dataset_config.split
#     )
#     seed = dataset_config.seed if dataset_config.seed is not None else global_seed
#     if dataset_config.streaming:
#         assert isinstance(dataset, IterableDataset)
#         # dataset = dataset.shuffle(seed=seed, buffer_size=buffer_size)
#     else:
#         # dataset = dataset.shuffle(seed=seed)
#         pass

#     tokenizer = AutoTokenizer.from_pretrained(dataset_config.tokenizer_name)
#     torch_dataset: Dataset
#     if dataset_config.is_tokenized:
#         torch_dataset = dataset.with_format("torch")  # type: ignore
#         # Get a sample from the dataset and check if it's tokenized and what the n_ctx is
#         # Note that the dataset may be streamed, so we can't just index into it
#         sample = next(iter(torch_dataset))[dataset_config.column_name]  # type: ignore
#         assert (
#             isinstance(sample, torch.Tensor) and sample.ndim == 1
#         ), "Expected the dataset to be tokenized."
#         assert len(sample) == dataset_config.n_ctx, "n_ctx does not match the tokenized length."

#     else:
#         torch_dataset = tokenize_and_concatenate(
#             dataset,  # type: ignore
#             tokenizer,
#             max_length=dataset_config.n_ctx,
#             add_bos_token=True,
#         )

#     # Note that a pre-tokenized dataset was shuffled when generated
#     # see e2e_sae.scripts.upload_hf_dataset.TextDataset.__init__
#     loader = DataLoader(
#         torch_dataset,  # type: ignore
#         batch_size=batch_size,
#         shuffle=False,
#     )
#     return loader, tokenizer

In [8]:
dataset_config  = DatasetConfig

dataset_config.model_config = None # blank
dataset_config.dataset_name= 'roneneldan/TinyStories' # 'apollo-research/roneneldan-TinyStories-tokenizer-gpt2'
dataset_config.is_tokenized= False # True
dataset_config.tokenizer_name= 'gpt2'
dataset_config.streaming= True # True - means you do not download the dataset https://huggingface.co/docs/datasets/en/stream
dataset_config.split= 'train' # ['train', 'validation']
dataset_config.n_ctx= 512
dataset_config.seed= 0
dataset_config.column_name: str = "input_ids"

In [9]:
device = 'cuda' # 'cpu', 'cuda'
model = HookedTransformer.from_pretrained("tiny-stories-1M").to(device)
dataset = load_dataset(
    dataset_config.dataset_name, streaming=dataset_config.streaming, split=dataset_config.split
)
# tokenizer = AutoTokenizer.from_pretrained(dataset_config.tokenizer_name) # 'gpt2' for tiny-stories-1M

c:\Users\MQTyor\ai_pc\ENVS_general\cmpttv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model tiny-stories-1M into HookedTransformer
Moving model to device:  cuda


In [11]:
"""
Gather hidden representations of the residual stream after the block  blocks.5
    blocks.5.hook_resid_post
"""

n_seqs = 3200 # 3200 # Num sequences to process
embs_list = []
tokens_list = []
text_id_list = []
seqs_saved = []
for i, v in tqdm(enumerate(dataset), total=n_seqs):
    # print(v)
    text_ = v['text']
    # print(text_)
    # tokenized = tokenizer(text_)['input_ids']
    # print(tokenized)
    # decoded = tokenizer.decode(tokenized)
    # print(decoded)
    tokenized = model.tokenizer(text_)['input_ids'] # No BOS token
    model.eval()
    with torch.no_grad():
        logits_, cache_ = model.run_with_cache(text_)
    acts_ = cache_['blocks.5.hook_resid_post'][0].cpu().numpy()[1:] # Skip BOS token
    # Append to lists
    embs_list.append(acts_)
    tokens_list.append(tokenized)
    text_id_list = text_id_list + [i]*len(acts_)
    seqs_saved.append(text_)
    if i >= n_seqs:
        break

 45%|████▌     | 1446/3200 [00:31<00:37, 46.45it/s]


KeyboardInterrupt: 

In [10]:
from pathlib import Path
import pandas as pd

embs_flat = np.array([v for vv in embs_list for v in vv])
tokens_str_list_flat = [model.tokenizer.decode(v) for vv in tokens_list for v in vv]
assert (len(embs_flat)==len(tokens_str_list_flat)==len(text_id_list)), 'Not equal len'

# Save embedding matrix
print(f"Embedding matrix takes {embs_flat.nbytes/1024**2:.3f} Mb")
datadir = Path ('./')
filepath_acts = datadir / f"emb_matrix.npy"
with open(filepath_acts, 'wb') as f:
    np.save(f, embs_flat)

df_acts_tokens = pd.DataFrame()
df_acts_tokens['emb_id'] = list(range(len(embs_flat)))
df_acts_tokens['token_str'] = tokens_str_list_flat
df_acts_tokens['seq_id'] = text_id_list
df_acts_tokens.emb_id = df_acts_tokens.emb_id.astype(np.int32)
# df_acts_tokens.token = df_acts_tokens.token.astype(np.int32)
df_acts_tokens.seq_id = df_acts_tokens.seq_id.astype(np.int32)

In [11]:
df_acts_tokens.head()

,emb_id,token_str,seq_id
0,0,One,0
1,1,day,0
2,2,",",0
3,3,a,0
4,4,little,0


In [12]:
# Play with sorting tokens by neuron activations
# qq = np.array(acts_list_flat)
# df_acts_tokens['nn'] = qq[:, 1]
# df_acts_tokens.sort_values(by='nn', ascending=False).head(20)

In [13]:
# Save the dataset of texts activations have been acquired from
df_texts = pd.DataFrame()
df_texts['seq'] = seqs_saved
df_texts['seq_id'] = df_texts.index

In [14]:
# # Left join activations dataframe with sequence dataframe
# df_acts_tokens.merge(df_texts, how='left', on='seq_id').info()

In [31]:
df_acts_tokens.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 682595 entries, 0 to 682594
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   emb_id     682595 non-null  int32 
 1   token_str  682595 non-null  object
 2   seq_id     682595 non-null  int32 
dtypes: int32(2), object(1)
memory usage: 10.4+ MB


In [32]:
df_texts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3201 entries, 0 to 3200
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   seq     3201 non-null   object
 1   seq_id  3201 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 50.1+ KB


In [17]:
df_acts_tokens.to_csv(datadir / "tinystories1M-TinyStories1_gpt2token-acts_id.csv", index=False)
df_texts.to_csv(datadir / "tinystories1M-TinyStories1_gpt2token-texts.csv", index=False)

In [ ]:
del df_acts_tokens
del df_texts
del embs_flat

In [311]:
# from pathlib import Path
# datadir = Path("./")

# # Load DataFrames with embedding ids and texts
# df_acts_filename = datadir / "tinystories1M-TinyStories1_gpt2token-acts_id.csv"
# df_texts_filename = datadir / "tinystories1M-TinyStories1_gpt2token-texts.csv"
# df_acts_tokens = pd.read_csv(df_acts_filename)
# df_texts= pd.read_csv(df_texts_filename)

# # Load embedding matrix
# with open(datadir / 'emb_matrix.npy', 'rb') as f:
#     embs_flat = np.load(f)
    
# # # Left join activations dataframe with sequence dataframe
# # df_acts_tokens_texts = df_acts_tokens.merge(df_texts, how='left', on='seq_id').info()